<a href="https://colab.research.google.com/github/zyberg2091/Dear-watson/blob/master/Dear_watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c contradictory-my-dear-watson

  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 85.3MB/s]
  0% 0.00/66.0k [00:00<?, ?B/s]
100% 66.0k/66.0k [00:00<00:00, 102MB/s]
  0% 0.00/536k [00:00<?, ?B/s]
100% 536k/536k [00:00<00:00, 171MB/s]


In [ ]:
# !unzip train.csv.zip -d sample_data/

In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('train.csv')
df.head()


,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [ ]:
df['language'].unique()

array(['English', 'French', 'Thai', 'Turkish', 'Urdu', 'Russian',
       'Bulgarian', 'German', 'Arabic', 'Chinese', 'Hindi', 'Swahili',
       'Vietnamese', 'Spanish', 'Greek'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12120 entries, 0 to 12119
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12120 non-null  object
 1   premise     12120 non-null  object
 2   hypothesis  12120 non-null  object
 3   lang_abv    12120 non-null  object
 4   language    12120 non-null  object
 5   label       12120 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 568.2+ KB


#Implementing transformer models and finding the best result

1. BERT-base-uncased-multilingual

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.9MB/s 
     |████████████████████████████████| 890kB 25.5MB/s 
     |████████████████████████████████| 3.2MB 34.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=36dd423c2ca48455cf4dbb3d0e1283084d11b14b56d5b8bccbadb904f72a4bea
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import transformers
from tokenizers import BertWordPieceTokenizer
from transformers import TFBertModel,BertTokenizerFast,BertTokenizer

In [ ]:
bert_tokenizer=BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
bert_tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.txt',
 './added_tokens.json')

In [ ]:
tokenizer=BertTokenizerFast('/content/vocab.txt')

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=119547, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
sentences=df['hypothesis'].values
prem_sentences=df['premise'].values

In [ ]:
df.label.nunique()

3

In [ ]:
labels=df['label'].values

label_data=tf.keras.utils.to_categorical(labels,num_classes=3)

In [ ]:
# no need of cleaning data..lets move on to preprocessing data before feeding into ML model pipeline
# data prep engine

class Bert_Engine:
  def __init__(self,max_len,spl_tokens,tokenizer):
    self.max_len=max_len
    self.tokenizer=tokenizer
    self.spl_tokens=spl_tokens

  def tokenization_and_padding(self,sentences,prem_sentences):
    hy_tokenized_sentences,prem_tokenized_sentences=[],[]
    for premise,sentence in zip(sentences,prem_sentences):
      hy_tokens=self.tokenizer.encode(sentence,add_special_tokens=self.spl_tokens,max_length=self.max_len,truncation=True)
      prem_tokens=self.tokenizer.encode(premise,add_special_tokens=self.spl_tokens,max_length=self.max_len,truncation=True)
      hy_tokenized_sentences.append(hy_tokens)
      prem_tokenized_sentences.append(prem_tokens)
    self.hy_tokenized_data=pad_sequences(hy_tokenized_sentences,maxlen=self.max_len)
    self.prem_tokenized_data=pad_sequences(prem_tokenized_sentences,maxlen=self.max_len)
    return self.hy_tokenized_data,self.prem_tokenized_data

  def attention_mask_and_padding(self):
    hy_attention_mask,prem_attention_mask=[],[]
    for hy_tokens,prem_tokens in zip(self.hy_tokenized_data,self.prem_tokenized_data):
      hy_masks=[1 if i>0 else 0 for i in hy_tokens]
      prem_masks=[1 if i>0 else 0 for i in prem_tokens]
      hy_attention_mask.append(hy_masks)
      prem_attention_mask.append(prem_masks)

    return hy_attention_mask,prem_attention_mask
        

In [ ]:
bert_eng_data=Bert_Engine(128,True,tokenizer)

In [ ]:
hy_bert_data,prem_bert_data=Bert_Engine.tokenization_and_padding(bert_eng_data,sentences,prem_sentences)
hy_attention_masks,prem_attention_masks=Bert_Engine.attention_mask_and_padding(bert_eng_data)


In [ ]:
#bert model

bert_layer=TFBertModel.from_pretrained('bert-base-multilingual-cased')


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
np.array(hy_bert_data).shape,np.array(hy_attention_masks).shape

((12120, 128), (12120, 128))

In [ ]:
a=bert_layer(hy_bert_data[:3])
a

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <tf.Tensor: shape=(3, 128, 768), dtype=float32, numpy=
                               array([[[-0.02316505, -0.07282426,  0.20971467, ...,  0.25082386,
                                         0.3252756 ,  0.00301654],
                                       [-0.02857281, -0.6237676 ,  1.7760389 , ...,  0.5345694 ,
                                         1.2395974 ,  0.37338266],
                                       [-0.01339581, -0.63685185,  1.8759284 , ...,  0.2495235 ,
                                         0.97389203,  0.03710479],
                                       ...,
                                       [-0.4549756 , -0.3377009 ,  0.3837086 , ...,  0.3248602 ,
                                        -0.22671485, -0.19689417],
                                       [-0.40761936, -0.19795358,  0.7960616 , ...,  0.13359071,
                                         0.2837106 , -0.01222535]

In [ ]:
import tensorflow as tf
a=bert_layer(hy_bert_data[:5])[0]
a[:,0,:]

<tf.Tensor: shape=(5, 768), dtype=float32, numpy=
array([[-0.02316505, -0.07282426,  0.20971467, ...,  0.25082386,
         0.3252756 ,  0.00301654],
       [ 0.03306587, -0.05607904,  0.23421305, ...,  0.18016726,
         0.335474  ,  0.04074974],
       [ 0.05850943,  0.01458559,  0.1452428 , ...,  0.1478333 ,
         0.22643188, -0.07473897],
       [-0.12675221, -0.04522049,  0.19922665, ...,  0.31033954,
         0.02512328,  0.03426644],
       [-0.07085975,  0.05138292,  0.00073937, ...,  0.11728048,
         0.02560398, -0.07689637]], dtype=float32)>

In [ ]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [ ]:
import tensorflow as tf

In [ ]:
for layer in bert_layer.layers:
  layer.trainable=False

In [ ]:
# model development
class Custom_Model_1(tf.keras.Model):
  def __init__(self,units,training):
    super(Custom_Model_1,self).__init__()
    self.units=units
    self.training=training
    self.dense_1=tf.keras.layers.Dense(self.units,activation='relu',trainable=self.training,name='dense_layer_1')
    self.bert1=bert_layer
  def call(self,input_ids,attention_mask):
    embedding1=self.bert1([input_ids,attention_mask])[0]
    cls1=embedding1[:,0,:]
    x=self.dense_1(cls1)
    
    return x



In [ ]:
class Custom_Model_2(tf.keras.Model):
  def __init__(self,units,training):
    super(Custom_Model_2,self).__init__()
    self.units=units
    self.training=training
    self.dense_2=tf.keras.layers.Dense(self.units,activation='relu',trainable=self.training,name='dense_layer_11')
    self.bert2=bert_layer

  def call(self,input_ids,attention_mask):
    embedding2=self.bert2([input_ids,attention_mask])[0]
    cls2=embedding2[:,0,:]
    x=self.dense_2(cls2)
    
    return x


In [ ]:
class Siamese_Model(tf.keras.Model):
    def __init__(self,units,training):
      super(Siamese_Model,self).__init__()
      self.units=units
      self.training=training
      self.out1=Custom_Model_1(self.units,self.training)
      self.out2=Custom_Model_2(self.units,self.training)
      self.dropout=tf.keras.layers.Dropout(0.2)
      self.dense_s=tf.keras.layers.Dense(256,activation='relu',trainable=self.training,name='dense_layer_s')
      self.dense=tf.keras.layers.Dense(3,activation='softmax',trainable=self.training,name='output_layer')

    def call(self,hy_inputs_ids,hy_attention_mask,prem_inputs_ids,prem_attention_mask):
      inp1=self.out1(hy_inputs_ids,hy_attention_mask)
      inp2=self.out2(prem_inputs_ids,prem_attention_mask)
      concat=tf.keras.layers.concatenate([inp1,inp2])
      x=self.dense_s(concat)
      x=self.dropout(x)
      x=self.dense(x)
      
      return x



In [ ]:
hy_inputs_ids,hy_attention_masks,prem_inputs_ids,prem_attention_masks = np.array(hy_bert_data), np.array(hy_attention_masks), np.array(prem_bert_data), np.array(prem_attention_masks)



In [ ]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [ ]:
model=Siamese_Model(256,True)
# f_log=model(hy_inputs_ids[:10],hy_attention_masks[:10],prem_inputs_ids[:10],prem_attention_masks[:10])  #simple un-trained logits

In [ ]:
model.trainable

True

In [ ]:
# model.summary()

In [ ]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
epoch_accuracy=tf.keras.metrics.CategoricalAccuracy()
optimizer=tf.keras.optimizers.SGD()
epoch_loss_avg = tf.keras.metrics.Mean()

In [ ]:
# batch_data=len(hy_inputs_ids)//32
# batch_data

In [ ]:
# tf.config.run_functions_eagerly(True)

In [ ]:
# tf.data 
# AUTO = tf.data.experimental.AUTOTUNE
# hy_inputs_ids, hy_attention_masks, prem_inputs_ids, prem_attention_masks, label_data = tf.convert_to_tensor(hy_inputs_ids), tf.convert_to_tensor(hy_attention_masks), tf.convert_to_tensor(prem_inputs_ids), tf.convert_to_tensor(prem_attention_masks), tf.convert_to_tensor(label_data)
# hy_data=tf.data.Dataset.from_tensor_slices(((hy_inputs_ids,hy_attention_masks),label_data)).batch(32,drop_remainder=True).prefetch(AUTO)
# prem_data=tf.data.Dataset.from_tensor_slices(((prem_inputs_ids,prem_attention_masks),label_data)).batch(32,drop_remainder=True).prefetch(AUTO)

In [ ]:
# for ((xh,yh),(xp,yp)) in zip(hy_data.take(2),prem_data.take(2)):
#   print(xh,xp)
  

In [ ]:
@tf.function
def custom_train(epochs, batches, hy_inputs_ids, hy_attention_masks, prem_inputs_ids, prem_attention_masks, label_data):
  train_loss,train_accuracy=[],[]
  
  for epoch in range(1,epochs+1):
    i=0
    batch_data=len(hy_inputs_ids)//batches
    for batch in range(batches):
      with tf.GradientTape() as tape:
        logits=model(hy_inputs_ids[i:batch_data+i],hy_attention_masks[i:batch_data+i],prem_inputs_ids[i:batch_data+i],prem_attention_masks[i:batch_data+i])
        loss_val=loss_fn(label_data[i:batch_data+i],logits)
      grad=tape.gradient(loss_val,model.trainable_weights)
      optimizer.apply_gradients(zip(grad,model.trainable_weights))

      tf.print('loss value after batch:',batch)
      tf.print('loss_value:',loss_val)
      epoch_accuracy.update_state(label_data[i:batch_data+i],logits)
      epoch_loss_avg.update_state(loss_val)
      i+=batch_data


    train_loss.append(epoch_loss_avg.result())
    train_accuracy.append(epoch_accuracy.result())

    # if epoch%5==0:
    tf.print('for epoch :',epoch)
    print('\n') 
    tf.print('training_loss :',epoch_loss_avg.result() ,'training_accuracy :',epoch_accuracy.result())
    print('\n')








In [ ]:
hy_inputs_ids, hy_attention_masks, prem_inputs_ids, prem_attention_masks, label_data = tf.convert_to_tensor(hy_inputs_ids), tf.convert_to_tensor(hy_attention_masks), tf.convert_to_tensor(prem_inputs_ids), tf.convert_to_tensor(prem_attention_masks), tf.convert_to_tensor(label_data)
custom_train(10, 32, hy_inputs_ids[:500], hy_attention_masks[:500], prem_inputs_ids[:500], prem_attention_masks[:500], label_data[:500])

NameError: ignored

In [ ]:
# model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

In [ ]:
# inputs=[hy_data,prem_data]
# ds=inputs[0].concatenate(inputs[1])

In [ ]:
# model.fit(ds,epochs=5)

Epoch 1/5
  5/756 [..............................] - ETA: 5:53:54 - loss: 1.1590 - accuracy: 0.2746